In [1]:
!pip install -qqq transformers datasets evaluate

In [2]:
import warnings
warnings.simplefilter("ignore")
import numpy as np 
import pandas as pd 
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback
from datasets import Dataset
import torch
import os
os.environ["WANDB_DISABLED"] = "true"

In [3]:
data_path = '/kaggle/input/clickbait-detection-msci641-s23'

In [4]:
train = pd.read_json(f'{data_path}/train.jsonl', lines=True)
test = pd.read_json(f'{data_path}/test.jsonl', lines=True)
val = pd.read_json(f'{data_path}/val.jsonl', lines=True)

In [5]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-large")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
def tokenize(examples):
    return tokenizer(examples["text"], truncation=True)

In [7]:
id2label = {0: 'passage', 1: 'phrase', 2: 'multi'}
label2id = {'passage': 0, 'phrase': 1, 'multi': 2}

In [8]:
def preprocess_data(df, test=False):
    ret = []
    if test:
        for _, i in df.iterrows():
            platform = i['postPlatform']
            post = ' '.join(i['postText'])
            title = i['targetTitle']
            description = i['targetDescription']
            paragraph = ' '.join(i['targetParagraphs'])
            keyword = i['targetKeywords']
            text = f"{platform} Post: {post} Website Title: {title} Website Description: {description} Website Paragraph: {paragraph} Website Keyword: {keyword}"
            ret += [{'text': text, 'id': i['id']}]
            ret_df = pd.DataFrame(ret)
        
    else:
        for _, i in df.iterrows():
            platform = i['postPlatform']
            post = ' '.join(i['postText'])
            title = i['targetTitle']
            description = i['targetDescription']
            paragraph = ' '.join(i['targetParagraphs'])
            keyword = i['targetKeywords']
            text = f"{platform} Post: {post} Website Title: {title} Website Description: {description} Website Paragraph: {paragraph} Website Keyword: {keyword}"
            ret += [{'text': text, 'labels': i['tags'][0]}]
            ret_df = pd.DataFrame(ret)
            ret_df['labels'] = ret_df['labels'].apply(lambda x: label2id[x])
    
    data = Dataset.from_pandas(ret_df)
    tokenized_data = data.map(tokenize, batched=True)
    return tokenized_data

In [9]:
train_df = preprocess_data(train)
val_df = preprocess_data(val)
test_df = preprocess_data(test, test=True)

  0%|          | 0/4 [00:00<?, ?ba/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [10]:
import evaluate
f1 = evaluate.load("f1", average='macro')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return f1.compute(predictions=predictions, references=labels, average='macro')

In [11]:
model = AutoModelForSequenceClassification.from_pretrained(
    "microsoft/deberta-v3-large", num_labels=3, id2label=id2label, label2id=label2id
)

Some weights of the model checkpoint at microsoft/deberta-v3-large were not used when initializing DebertaV2ForSequenceClassification: ['mask_predictions.classifier.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.LayerNorm.bias', 'mask_predictions.LayerNorm.weight', 'mask_predictions.dense.bias', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.LayerNorm.weight']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from 

In [12]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [13]:
training_args = TrainingArguments(
    output_dir="model",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    fp16=True,
    optim="adafactor",
    num_train_epochs=50,
    warmup_ratio=0.1,
    weight_decay=0.01,
    logging_steps=5,
    evaluation_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_df,
    eval_dataset=val_df,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=5)],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [15]:
trainer.train()

You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 trainer.train()                                                                              │
│   2                                                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/trainer.py:1645 in train                    │
│                                                                                                  │
│   1642 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1643 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1644 │   │   )                                                                                 │
│ ❱ 1645 │   │   return inner_training_loop(                                                       │
│   1646 │   │   │   args=args,                                                                    │
│   1647 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1648 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/trainer.py:2020 in _inner_training_loop     │
│                                                                                                  │
│   2017 │   │   │   │   │   self.state.epoch = epoch + (step + 1 + steps_skipped) / steps_in_epo  │
│   2018 │   │   │   │   │   self.control = self.callback_handler.on_step_end(args, self.state, s  │
│   2019 │   │   │   │   │                                                                         │
│ ❱ 2020 │   │   │   │   │   self._maybe_log_save_evaluate(tr_loss, model, trial, epoch, ignore_k  │
│   2021 │   │   │   │   else:                                                                     │
│   2022 │   │   │   │   │   self.control = self.callback_handler.on_substep_end(args, self.state  │
│   2023                                                                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/trainer.py:2321 in _maybe_log_save_evaluate │
│                                                                                                  │
│   2318 │   │   │   │   │   )                                                                     │
│   2319 │   │   │   │   │   metrics.update(dataset_metrics)                                       │
│   2320 │   │   │   else:                                                                         │
│ ❱ 2321 │   │   │   │   metrics = self.evaluate(ignore_keys=ignore_keys_for_eval)                 │
│   2322 │   │   │   self._report_to_hp_search(trial, self.state.global_step, metrics)             │
│   2323 │   │   │                                                                                 │
│   2324 │   │   │   # Run delayed LR scheduler now that metrics are populated                     │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/trainer.py:3053 in evaluate                 │
│                                                                                                  │
│   3050 │   │   start_time = time.time()                                                          │
│   3051 │   │                                               

In [ ]:
pred = trainer.predict(test_df).predictions
pred_ids = np.argmax(pred, 1)
test['spoilerType'] = pred_ids
test['spoilerType'] = test['spoilerType'].apply(lambda x: id2label[x])
submissions = test[['id', 'spoilerType']]

In [ ]:
submissions.to_csv('submissions.csv', index=False)